# Computing Visions Jupyter Notebook

# Allie Garrison


In [ ]:
import sqlite3
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
conn = sqlite3.connect('zippedData/im.db')

bom_movie_gross = pd.read_csv("zippedData/bom.movie_gross.csv")
rt_movie_info = pd.read_csv("zippedData/rt.movie_info.tsv", sep='\t')
rt_reviews = pd.read_csv('zippedData/rt.reviews.tsv', sep='\t',encoding='windows-1252')
tmdb_movies = pd.read_csv("zippedData/tmdb.movies.csv")
tn_movie_budgets = pd.read_csv("zippedData/tn.movie_budgets.csv", parse_dates=['release_date'])

In [ ]:
## Making copy of movie budgets data frame for cleaning
tn_movie_budgets_clean = tn_movie_budgets.copy()

## turning the columns with dollar amounts into int types in millions of dollars
tn_movie_budgets_clean['production_budget_in_millions'] = tn_movie_budgets_clean['production_budget'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['production_budget_in_millions'] = tn_movie_budgets_clean['production_budget_in_millions'].astype('int64') / 1000000

tn_movie_budgets_clean['domestic_gross_in_millions'] = tn_movie_budgets_clean['domestic_gross'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['domestic_gross_in_millions'] = tn_movie_budgets_clean['domestic_gross_in_millions'].astype('int64') / 1000000

tn_movie_budgets_clean['worldwide_gross_in_millions'] = tn_movie_budgets_clean['worldwide_gross'].str.strip('$').str.replace(',', '')
tn_movie_budgets_clean['worldwide_gross_in_millions'] = tn_movie_budgets_clean['worldwide_gross_in_millions'].astype('int64') / 1000000

## taking out all movies that were released before January 1, 2010
tn_movie_budgets_clean = tn_movie_budgets_clean[tn_movie_budgets_clean['release_date'] >= "2010-01-01"]

## taking out all movies that did not make a profit
tn_movie_budgets_clean = tn_movie_budgets_clean[tn_movie_budgets_clean['domestic_gross_in_millions'] != 0]

In [ ]:
## Creating columns with return on investment (ROI) for domestic and international revenue
tn_movie_budgets_clean['ROI_domestic'] = (tn_movie_budgets_clean['domestic_gross_in_millions'] / tn_movie_budgets_clean['production_budget_in_millions']) * 100
tn_movie_budgets_clean['ROI_international'] = (tn_movie_budgets_clean['worldwide_gross_in_millions'] / tn_movie_budgets_clean['production_budget_in_millions']) * 100

### 

In [ ]:
## joining movie_basics and movie_ratings to create a data frame with title, runtime, genre, ratings, and votes
table_genre = pd.read_sql('''
SELECT mb.primary_title, 
        mb.runtime_minutes, 
        mb.genres, 
        mr.averagerating,
        mr.numvotes
FROM movie_basics AS mb
JOIN movie_ratings AS mr USING (movie_id)
''', conn)

In [ ]:
## merging the cleaned tn.movie_budgets dataframe with table_genre on movie title
genre_ratings_budget = tn_movie_budgets_clean.merge(table_genre, how='inner', left_on='movie', right_on='primary_title')

In [ ]:
genre_ratings_budget.head()

In [ ]:
## creating a dataframe with only the movies with animation as a genre
animation = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Animation') == True]
avg_rating = genre_ratings_budget['profit_in_millions'].mean()

# running a 1-sample t-test on the animation profit vs. population profit
stats.ttest_1samp(action['profit_in_millions'], popmean=avg_rating)


# Cole Sussmeier

In [ ]:
# create profit column
genre_ratings_budget['profit_in_millions'] = genre_ratings_budget['worldwide_gross_in_millions'] - genre_ratings_budget['production_budget_in_millions']

In [ ]:
plt.title("Distribution of Total Profit for all Movies")
plt.xlabel("Profit in Millions")
plt.ylabel("Number of Movies")
sns.histplot(genre_ratings_budget['profit_in_millions'], bins=50);

### Get average values for features by genre

In [ ]:
# genre_hash is a dictionary where each key is a genre, and the value is the number of times it occurs in the 'genres' column
genre_hash = {}

# array of lists where each individual list is every genre listed for a single movie
genre_lists = genre_ratings_budget['genres'].str.split(',')

for i, genres in enumerate(genre_lists):
    # try/ except in case there is an empty list (no genres listed for a movie)
    try:
        for genre in genres:
            # add one to genre if the key exists
            if genre in genre_hash.keys():
                genre_hash[genre] += 1
            # otherwise create the key
            else:
                genre_hash[genre] = 1
    except:
        pass


In [ ]:
genre_hash

In [ ]:
genre_stats = []
for genre in genre_hash.keys():
    #filter for every entry that contains the current genre
    mask = genre_ratings_budget[genre_ratings_budget['genres'].str.contains(genre) == True]
    # append list with averages for relevant features
    genre_stats.append([genre, mask['profit_in_millions'].mean().round(2), 
                        mask['production_budget_in_millions'].mean().round(2),
                        mask['averagerating'].mean().round(2),
                        mask['numvotes'].mean().round(2)])

In [ ]:
#convert to dataframe
genre_stats_df = pd.DataFrame(genre_stats, 
                              columns=['Genre', 'Average_Profit', 'Average_Budget', 'Average_Rating', 'Average_Votes'])
genre_stats_df

In [ ]:
# important note: There are many instances where genres overlap, ie Avengers Endgame is action and adventure.
# This means that the profit of this individual movie is factored into the averages for both genres 
# and they are not independent measures

In [ ]:
plt.figure(figsize=(25,15))
sns.set(font_scale=2)
plt.xticks(rotation=45)
plt.title("Average Profit by Genre")
sns.barplot(data = genre_stats_df, x='Genre', y='Average_Profit');


In [ ]:
plt.figure(figsize=(25,15))
plt.xticks(rotation=45)
plt.title("Average Budget by Genre")
sns.barplot(data = genre_stats_df, x='Genre', y='Average_Budget');

In [ ]:
plt.figure(figsize=(25,15))
plt.xticks(rotation=45)
plt.title("Average Rating by Genre")
sns.barplot(data = genre_stats_df, x='Genre', y='Average_Rating');

In [ ]:
plt.figure(figsize=(25,15))
plt.xticks(rotation=45)
plt.title("Average Votes by Genre")
sns.barplot(data = genre_stats_df, x='Genre', y='Average_Votes');

In [ ]:
plt.title("Correlation Plot")
sns.heatmap(genre_stats_df.corr().round(2), cmap="RdYlGn", annot=True);

### Correlation plot for animation movies

In [ ]:
animation = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Animation') == True]

In [ ]:
sns.set(font_scale=1)
sns.relplot(data=animation, 
            x ="production_budget_in_millions", y="profit_in_millions")
plt.xlabel("Budget (Millions USD)")
plt.ylabel("Profit (Millions USD)")
plt.title("Budget vs Profit for Animation Movies");

In [ ]:
sns.set(font_scale=1)
sns.relplot(data=animation, 
            x ="production_budget_in_millions", y="averagerating")
plt.xlabel("Budget (Millions USD)")
plt.ylabel("Average Rating")
plt.title("Budget vs Average Rating for Animation Movies");

In [ ]:
# top ten most profitable
animation.sort_values('profit_in_millions', ascending=False).head(10)

In [ ]:
# top ten animated movies by rating
animation.sort_values('averagerating', ascending=False).head(10)

In [ ]:
# feel free to run any statistical tests that may be relevant to our reccomendations... this is just the outline required
# for a 1sample t-test

In [ ]:
avg_profit = genre_ratings_budget['profit_in_millions'].mean()
print(avg_profit)

In [ ]:
action = genre_ratings_budget[genre_ratings_budget['genres'].str.contains('Action') == True]

In [ ]:
from scipy import stats

In [ ]:
stats.ttest_1samp(action['profit_in_millions'], popmean=avg_profit)

In [ ]:
# avg profit from action is significantly different than avg profit overall

## Working Area 1- Jacob and Matt

## Creating a list of the top 10 animation movies by runtime



To recommend a runtime we took our suggested genre of animation and wanted to see the runtimes of the highest rated animated films so we could emulate their lengths.

In [ ]:
#Querying to find the runtime of the top 10 animation movies by rating 
df_runtime_raw = pd.read_sql('''
SELECT 
    primary_title as "Movie", runtime_minutes as "Run Time"
FROM 
    movie_basics as mb
JOIN
    movie_ratings USING (movie_id)
where 
    genres like "%Animation%"
    and  "Run Time" < 200
AND 
    runtime_minutes != "Nan"
ORDER BY
    averagerating DESC
LIMIT 10


''', conn)

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.barplot(data = df_runtime_raw, x='Movie', y='Run Time')
plt.xlabel("Movie")
plt.ylabel("Run Time")
plt.title("Animation Movies Run Time")
plt.xticks(rotation = 45)
#plotting our movies from above

<h2> Working Area 2 - Cameron Narimanian

<h8> Due to JSON merging errors corrupting the main 'Primary Notebook' everytime I would push, (this happened like 3 times where we had to reset the main notebook), we decided to push it from Matt's python set.  Cam sent the code set to Matt to push to see if it make a difference, just as a temporary fix, we know it is not in best practice however given our timeframe and resources (not knowing JSON and consulting instructors) we decided this was the best alternative for the amount of time we had

In [ ]:
<h3> Number of Films Per Genre Since 2010

In [ ]:
genre_stats_df.sort_values('Average_Profit',ascending= False)

### Based on average profit since 2010, Animation has the highest profit

<h4> Top 10 Animation Movies Based on Profits in Millions since 2010

In [ ]:
df_topanimation=animation.sort_values('profit_in_millions').head(10)

df_topanimation

<h5> Top Animation Runtime in Minutes

In [ ]:
df_topanimation['runtime_minutes']

<h4> Boxplot Top Animation Movies based on Profit

In [ ]:
## BASED OFF THE MOVIE PROFITS

plt.subplots(figsize=(5, 5))

df_topanimation['runtime_minutes']

sns.boxplot(data= df_topanimation , y = 'runtime_minutes')

plt.xlabel("Movies")
plt.ylabel("Run Time")
plt.title("Top Animation Movies Run Time")

<h5> 25% of the animation movies are between 87 minutes long and 102 minutes long with an average runtime of 91 minutes 

In [ ]:
df_topanimation['runtime_minutes'].describe()

Due to JSON merging errors corrupting the main 'Primary Notebook' everytime I would push, (this happened like 3 times where we had to reset the main notebook), we decided to push it from Matt's python set.  Cam sent the code set to Matt to push to see if it make a difference, just as a temporary fix, we know it is not in best practice however given our timeframe and resources (not knowing JSON and consulting instructors) we decided this was the best alternative for the amount of time we had

## Number of Films Per Genre Since 2010

In [ ]:
genre_stats_df.sort_values('Average_Profit',ascending= False)

### Based on average profit since 2010, Animation has the highest profit

## Top 10 Animation Movies Based on Profits in Millions since 2010

In [ ]:
df_topanimation=animation.sort_values('profit_in_millions').head(10)

df_topanimation

## Top Animation Runtime in Minutes

In [ ]:
df_topanimation['runtime_minutes']

<h4> Boxplot Top Animation Movies based on Profit

In [ ]:
## BASED OFF THE MOVIE PROFITS

plt.subplots(figsize=(5, 5))

df_topanimation['runtime_minutes']

sns.boxplot(data= df_topanimation , y = 'runtime_minutes')

plt.xlabel("Movies")
plt.ylabel("Run Time")
plt.title("Top Animation Movies Run Time")

<h5> 25% of the animation movies are between 87 minutes long and 102 minutes long with an average runtime of 91 minutes 

In [ ]:
df_topanimation['runtime_minutes'].describe()

In [ ]:
sns.set(font_scale=1)
sns.relplot(data=genre_stats_df, 
            x ="Average_Profit", y="Average_Budget",
            hue = "Genre")
scifi = genre_stats_df[genre_stats_df['Genre'] == 'Sci-Fi']
animation = genre_stats_df[genre_stats_df['Genre'] == 'Animation']
musical = genre_stats_df[genre_stats_df['Genre'] == 'Musical']
music = genre_stats_df[genre_stats_df['Genre'] == 'Music']
plt.text(scifi['Average_Profit']-15, scifi['Average_Budget']+3, 'Sci Fi')
plt.text(animation['Average_Profit']-30, animation['Average_Budget']+3, 'Animation')
plt.text(musical['Average_Profit']-20, musical['Average_Budget']+3, 'Musical')
plt.text(western['Average_Profit']-20, music['Average_Budget']+3, 'Music')



plt.xlabel("Average Profit (Millions USD)")
plt.ylabel("Average Budget (Millions USD)")
plt.title("Average Budget vs Profit per Genre of Movies");

In [ ]:
sns.set(font_scale=1)
sns.relplot(data=genre_stats_df, 
            x ="Average_Profit", y="Average_Budget",
            hue = "Genre")
scifi = genre_stats_df[genre_stats_df['Genre'] == 'Sci-Fi']
animation = genre_stats_df[genre_stats_df['Genre'] == 'Animation']
musical = genre_stats_df[genre_stats_df['Genre'] == 'Musical']
music = genre_stats_df[genre_stats_df['Genre'] == 'Music']
plt.text(scifi['Average_Profit']-15, scifi['Average_Budget']+3, 'Sci Fi')
plt.text(animation['Average_Profit']-30, animation['Average_Budget']+3, 'Animation')
plt.text(musical['Average_Profit']-20, musical['Average_Budget']+3, 'Musical')
plt.text(western['Average_Profit']-20, music['Average_Budget']+3, 'Music')



plt.xlabel("Average Profit (Millions USD)")
plt.ylabel("Average Budget (Millions USD)")
plt.title("Average Budget vs Profit per Genre of Movies");

## Working Area 3